In [2]:
import random
import numbers
from PIL import Image, ImageMath
import os
import os.path
import numpy as np
import struct
import math

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

import time
import itertools

import pcl
import timeit
import png

import multiprocessing

from plyfile import PlyData, PlyElement

In [4]:
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:, 1] - extents[:, 0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize / 2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)


def plot_pc(pc_np, z_cutoff=1000, birds_view=False, color='height', size=0.3, ax=None, cmap=cm.jet, is_equal_axes=True):
    # remove large z points
    valid_index = pc_np[:, 0] < z_cutoff
    pc_np = pc_np[valid_index, :]

    if ax is None:
        fig = plt.figure(figsize=(9, 9))
        ax = Axes3D(fig)
    if type(color)==str and color == 'height':
        c = pc_np[:, 2]
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=c, cmap=cmap, edgecolors='none')
    elif type(color)==str and color == 'reflectance':
        assert False
    elif type(color) == np.ndarray:
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=color, cmap=cmap, edgecolors='none')
    else:
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=color, edgecolors='none')

    if is_equal_axes:
        axisEqual3D(ax)
    if True == birds_view:
        ax.view_init(elev=0, azim=-90)
    else:
        ax.view_init(elev=-45, azim=-90)
    # ax.invert_yaxis()

    return ax

# get surface normal
def Surface_normals(cloud):
    ne = cloud.make_NormalEstimation()
    tree = cloud.make_kdtree()
    ne.set_SearchMethod(tree)
#     ne.set_RadiusSearch(2)
    ne.set_KSearch(9)
    cloud_normals = ne.compute()
    return cloud_normals

In [9]:
dataset_root = '/ssd/jiaxin/TSF_datasets/3DMatch_eval'
output_root = '/ssd/jiaxin/TSF_datasets/3DMatch_eval_npy'
scene_list = ['7-scenes-redkitchen', 'sun3d-home_at-home_at_scan1_2013_jan_1', 
              'sun3d-home_md-home_md_scan9_2012_sep_30', 'sun3d-hotel_uc-scan3',
              'sun3d-hotel_umd-maryland_hotel1', 'sun3d-hotel_umd-maryland_hotel3', 
              'sun3d-mit_76_studyroom-76-1studyroom2', 'sun3d-mit_lab_hj-lab_hj_tea_nov_2_2012_scan1_erika']
downsample_size = 16384

for scene in scene_list:
    ply_filename_list = os.listdir(os.path.join(dataset_root, scene))
    
    npy_folder = os.path.join(output_root, scene)
    if not os.path.isdir(npy_folder):
        os.makedirs(npy_folder)
    
    for ply_filename in ply_filename_list:
        plydata = PlyData.read(os.path.join(dataset_root, scene, ply_filename))
        ply_x = plydata['vertex']['x']
        ply_y = plydata['vertex']['y']
        ply_z = plydata['vertex']['z']
        pc_np = np.stack((ply_x, ply_y, ply_z), axis=1)
        
        # downsample
        if pc_np.shape[0] > downsample_size:
            choice_idx = np.random.choice(pc_np.shape[0], downsample_size, replace=False)
            pc_np = pc_np[choice_idx, :]
            
        # compute surface normal
        cloud = pcl.PointCloud(pc_np.astype(np.float32))    
        sn = Surface_normals(cloud)
        sn_np = np.asarray(sn.to_array(), dtype=np.float32)  # Nx4, nx,ny,nz,curvature

        output_np = np.concatenate((pc_np.astype(np.float32), sn_np), axis=1)  # Nx7
        
        npy_filename = ply_filename[0:-4] + '.npy'
        npy_file = os.path.join(npy_folder, npy_filename)
        np.save(npy_file, output_np)
#         break;